In [10]:
import os
import cv2
import glob
import h5py
import shutil
import imgaug as aug
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import imgaug.augmenters as iaa
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from pathlib import Path
from skimage.io import imread
from skimage.transform import resize
from keras.models import Sequential, Model, load_model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten,Activation
from keras.models import Sequential
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


color = sns.color_palette()
%matplotlib inline
%config InlineBackend.figure_format="svg"

import tensorflow as tf

In [11]:
# Set the seed for hash based operations in python
os.environ['PYTHONHASHSEED'] = '0'

seed=1234

# Set the numpy seed
np.random.seed(seed)

# Set the random seed in tensorflow at graph level
tf.set_random_seed(seed)

# Make the augmentation sequence deterministic
aug.seed(seed)

In [12]:
training_data = Path('training') 
validation_data = Path('validation') 
labels_path = Path('monkey_labels.txt')

In [13]:
monkey_labels = []

# Read the file
lines = labels_path.read_text().strip().splitlines()[1:]

In [14]:
lines

['n0   , alouatta_palliata\t , mantled_howler                , 131          , 26',
 'n1   , erythrocebus_patas\t , patas_monkey                  , 139          , 28',
 'n2   , cacajao_calvus\t     , bald_uakari                   , 137          , 27',
 'n3   , macaca_fuscata\t     , japanese_macaque              , 152          , 30',
 'n4   , cebuella_pygmea\t     , pygmy_marmoset                , 131          , 26',
 'n5   , cebus_capucinus\t     , white_headed_capuchin         , 141          , 28',
 'n6   , mico_argentatus\t     , silvery_marmoset              , 132          , 26',
 'n7   , saimiri_sciureus\t     , common_squirrel_monkey        , 142          , 28',
 'n8   , aotus_nigriceps\t     , black_headed_night_monkey     , 133          , 27',
 'n9   , trachypithecus_johnii , nilgiri_langur                , 132          , 26']

In [15]:
for line in lines:
    line = line.split(',')
    line = [x.strip(' \n\t\r') for x in line]
    line[3], line[4] = int(line[3]), int(line[4])
    line = tuple(line)
    monkey_labels.append(line)

In [16]:
monkey_labels = pd.DataFrame(monkey_labels, columns=['Label', 'Latin Name', 'Common Name','Train Images', 'Validation Images'], index=None)


In [17]:
monkey_labels

,Label,Latin Name,Common Name,Train Images,Validation Images
0,n0,alouatta_palliata,mantled_howler,131,26
1,n1,erythrocebus_patas,patas_monkey,139,28
2,n2,cacajao_calvus,bald_uakari,137,27
3,n3,macaca_fuscata,japanese_macaque,152,30
4,n4,cebuella_pygmea,pygmy_marmoset,131,26
5,n5,cebus_capucinus,white_headed_capuchin,141,28
6,n6,mico_argentatus,silvery_marmoset,132,26
7,n7,saimiri_sciureus,common_squirrel_monkey,142,28
8,n8,aotus_nigriceps,black_headed_night_monkey,133,27
9,n9,trachypithecus_johnii,nilgiri_langur,132,26
